In [741]:
%matplotlib ipympl
from QDP import qdp
import numpy as np
import matplotlib.pyplot as plt
import os.path
dp = '/home/ebert/jupyter-notebooks/FNode-data'
exclude = [2]
drop_itr = [0,1,2,3,4,5]

In [705]:
exp_name = '2018_07_19_20_35_41_csrb-collisions-31-hd'
if exp_name:
    exp_date = '_'.join(exp_name.split('_')[:3])
    experiment_file = os.path.join(exp_date, exp_name, 'results.hdf5')
else:
    experiment_file = ''
q = qdp.QDP(base_data_path=dp)
q.load_data_file(experiment_file)
print "ivars: {}".format(q.experiments[0]['variable_list'])
roi_map = ['cs', 'rb']
cs_cuts = [[23],[3]]
rb_cuts = [[23],[3]]
q.set_thresholds(cs_cuts, roi=0)
q.set_thresholds(rb_cuts, roi=1)
retention = q.apply_thresholds(exclude_rois=exclude)
tbl_str = "ivar:\t{0}\tretention:{1} +- {2}"
entries = np.product(retention['retention'].shape[:-1])
for i in range(entries):
    print(tbl_str.format(
        retention['ivar'].reshape((entries,-1))[i][0],
        retention['retention'].reshape((entries,-1))[i],
        retention['error'].reshape((entries,-1))[i]
    ))
# q.save_experiment_data()
q.experiments[0]['variable_list']
try:
    xlab = q.experiments[0]['variable_desc'][q.experiments[0]['variable_list'][0]]['description']
except IndexError:
    print("Only one iteration")
    xlab = ''

fig, ax = plt.subplots()
for e in range(len(retention['ivar'])):
    for r in range(2):
        ax.errorbar(
            retention['ivar'][e, :, r], retention['retention'][e, :, r],
            yerr=retention['error'][e, :, r],
            fmt=['bo','rx'][r]
        )
ax.set_ylim(0, 1.01)
ax.set_xlabel(xlab)
ax.set_ylabel('F=4(2) pop.')

fmts = ['pdf', 'png']
fn = os.path.join(dp, q.experiments[0]['source_path'], 'retention_scan.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

data at: 2018_07_19/2018_07_19_20_35_41_csrb-collisions-31-hd/results.hdf5
name 'fort_exp' is not defined
name 'rb_uwave_freq' is not defined
ivars: [u'gap_time']
ivar:	8.5	retention:[ 0.73639456  0.84636872         nan] +- [ 0.01133337  0.01133337  0.01133337]
ivar:	30.0	retention:[ 0.65366615  0.78350515         nan] +- [ 0.01329247  0.01329247  0.01329247]
ivar:	60.0	retention:[ 0.62919897  0.79112754         nan] +- [ 0.0146981  0.0146981  0.0146981]
ivar:	120.0	retention:[ 0.65696784  0.85215054         nan] +- [ 0.01547721  0.01547721  0.01547721]
ivar:	300.0	retention:[ 0.53989362  0.84730539         nan] +- [ 0.01170414  0.01170414  0.01170414]
ivar:	600.0	retention:[ 0.64516129  0.80763583        -inf] +- [ 0.01270259  0.01270259  0.01270259]
ivar:	1200.0	retention:[ 0.66430595  0.75              -inf] +- [ 0.01396504  0.01396504  0.01396504]


FigureCanvasNbAgg()

In [742]:
dual_data = np.zeros((2, len(q.experiments[0]['iterations'].keys()) - len(drop_itr)))
indv_data = np.zeros((2, len(q.experiments[0]['iterations'].keys()) - len(drop_itr)))
dual_err = np.zeros((2, len(q.experiments[0]['iterations'].keys()) - len(drop_itr)))
indv_err = np.zeros((2, len(q.experiments[0]['iterations'].keys()) - len(drop_itr)))
dual_ret_data = []
j = 0

for i in q.experiments[0]['iterations'].keys():
    if i in drop_itr:
        print('Skipping iteration: {}'.format(i))
        j += 1
        continue
    results = q.generate_thresholds(itr=i, loss=False)
    itr_cuts = {
        0: q.get_thresholds()[0],
        1: q.get_thresholds()[1]
    }
    print(itr_cuts)
    q.apply_thresholds(exclude_rois=exclude)
    qd = q.experiments[0]['iterations'][i]['quantized_data']
    dual_load = np.logical_and(qd[:,0,0], qd[:,0,1])
    print(np.sum(dual_load))
    cs_only = np.logical_and(qd[:,0,0], np.logical_not(qd[:,0,1]))
    rb_only = np.logical_and(np.logical_not(qd[:,0,0]), qd[:,0,1])
    dual_ret =  np.mean(qd[dual_load,1], axis=0)[:2]
    cs_ret = np.mean(qd[cs_only,1], axis=0)[:2]
    rb_ret = np.mean(qd[rb_only,1], axis=0)[:2]
    
    dual_ret_data.append(qd[dual_load,1])

    dual_data[:, i-j] = dual_ret
    indv_data[0, i-j] = cs_ret[0]
    indv_data[1, i-j] = rb_ret[1]

    n_dual = sum(dual_load)
    n_cs = sum(cs_only)
    n_rb = sum(rb_only)

    def binomial_err(n, p):
        return np.sqrt(p*(1-p)/n)

    dual_err[:, i-j] = binomial_err(n_dual, np.mean(qd[dual_load,1], axis=0))[:2]
    cs_err = binomial_err(n_cs, np.mean(qd[cs_only,1], axis=0))[0]
    rb_err = binomial_err(n_rb, np.mean(qd[rb_only,1], axis=0))[1]
    indv_err[0, i-j] = cs_err
    indv_err[1, i-j] = rb_err

Skipping iteration: 0
Skipping iteration: 1
Skipping iteration: 2
Skipping iteration: 3
Skipping iteration: 4
Skipping iteration: 5
--------------------
--------------------
--------------------
--------------------
On entry to DGESDD parameter number 5 had an illegal value
There may be some issue with your guess: `[  3.70485993e-18   0.00000000e+00   0.00000000e+00]`
--------------------
On entry to DGESDD parameter number 5 had an illegal value
There may be some issue with your guess: `[  3.70485993e-18   0.00000000e+00   0.00000000e+00]`
--------------------
{0: [[33], [2]], 1: [[32], [4]]}
147


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:40: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in sqrt


In [743]:
dual_data

array([[ 0.74829932],
       [ 0.84353741]])

In [744]:
indv_data

array([[ 0.86078886],
       [ 0.84529148]])

In [745]:
fig, ax = plt.subplots(ncols=2)
for r in range(2):
    ax[r].errorbar(
        np.arange(dual_data.shape[1]), dual_data[r],
        yerr=dual_err[r],
        fmt=['bo','rx'][r],
        label=['cs(rb)', 'rb(cs)'][r]
    )
    ax[r].errorbar(
        0.2+np.arange(dual_data.shape[1]), indv_data[r],
        yerr=indv_err[r],
        fmt=['cs','yd'][r],
        label=['cs(!rb)', 'rb(!cs)'][r]
    )
    # plot weighted means
    indv_wmean = np.dot(indv_data[r], np.power(indv_err[r], -2))/np.sum(np.power(indv_err[r], -2))
    indv_stderr = np.sqrt(np.sum(np.power(indv_err[r], 2)))/dual_data.shape[1]
    dual_wmean = np.dot(dual_data[r], np.power(dual_err[r], -2))/np.sum(np.power(dual_err[r], -2))
    dual_stderr = np.sqrt(np.sum(np.power(dual_err[r], 2)))/dual_data.shape[1]
    print ("[{},{},{},{}]".format(indv_wmean, indv_stderr, dual_wmean, dual_stderr))
    ax[r].axhline(indv_wmean, color='cy'[r], alpha=0.5, lw=3)
    ax[r].axhline(dual_wmean, color='br'[r], alpha=0.5, lw=3)
    # plot uncertainty of the mean
    ax[r].fill_between([0, dual_data.shape[1]], indv_wmean-indv_stderr, indv_wmean+indv_stderr, facecolor='cy'[r], alpha=0.2)
    ax[r].fill_between([0, dual_data.shape[1]], dual_wmean-dual_stderr, dual_wmean+dual_stderr, facecolor='br'[r], alpha=0.2)
    
    ax[r].set_xlabel(xlab)
    ax[r].legend(bbox_to_anchor=(0.95, 0.15), loc=1, borderaxespad=0.)
    ax[r].set_ylim(0., 1)
ax[0].set_ylabel('Retention')

fmts = ['pdf', 'png']
fn = os.path.join(dp, q.experiments[0]['source_path'], 'dual_retention_stability.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

FigureCanvasNbAgg()

[0.860788863109,0.0166742640558,0.748299319728,0.0357949040403]
[0.845291479821,0.0171235155465,0.843537414966,0.0299639289403]


In [746]:
ps = np.zeros((2,2))
n = 0
for itr in dual_ret_data:
    n += len(itr)
    ps[1,1] += np.sum(np.logical_and(itr[:,0], itr[:,1]))  # both atoms retained
    ps[0,0] += np.sum(np.logical_and(np.logical_not(itr[:,0]), np.logical_not(itr[:,1])))  # both atoms lost
    ps[0,1] += np.sum(np.logical_and(np.logical_not(itr[:,0]), itr[:,1]))  # cs only lost
    ps[1,0] += np.sum(np.logical_and(itr[:,0], np.logical_not(itr[:,1])))  # cs only lost
ps = ps.astype('float')/n

In [747]:
print('['+','.join(['[{},{}]'.format(ps[i][0],ps[i][1]) for i in range(2)])+']')

[[0.102040816327,0.149659863946],[0.0544217687075,0.69387755102]]


In [748]:
print('['+','.join(['[{},{}]'.format(binomial_err(n, ps)[i][0],binomial_err(n, ps)[i][1]) for i in range(2)])+']')

[[0.0249664384425,0.0294232298746],[0.0187101121842,0.0380128420128]]


In [749]:
binomial_err(n,1./(2*n))

0.0048020625462823433